In [18]:
from selenium import webdriver
import time, re, imaplib, email
import base64
from bs4 import BeautifulSoup
import random

import psycopg2
conn = psycopg2.connect(user="postgres", password="postgres", host="127.0.0.1", port="5432", database="postgres")
cur = conn.cursor()

import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']
creds = None
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server()
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)
service = build('gmail', 'v1', credentials=creds)

def get_code_from_gmail():
    results = service.users().messages().list(userId='me',labelIds = ['INBOX']).execute()
    messages = results.get('messages', [])
    if not messages:
        print("No messages found.")
    else:
        # https://developers.google.com/gmail/api/v1/reference/users/messages?refresh=1
        for message in messages:
            msg = service.users().messages().get(userId='me', id=message['id']).execute()
            body = msg['payload']['parts'][1]['body']['data']
            contents = base64.urlsafe_b64decode(body)
            soup = BeautifulSoup(contents, 'html.parser')
            return(int(soup.find_all('h2')[1].text.strip()))
        
def get_code_from_hotmail():
    mail = imaplib.IMAP4_SSL('smtp-mail.outlook.com')
    mail.login('rajatpundir13@hotmail.com', '1nst1nct')
    mail.list()
    mail.select("inbox")
    result, data = mail.search(None, "ALL")
    ids = data[0]
    id_list = ids.split()
    latest_email_id = id_list[-1]
    result, data = mail.fetch(latest_email_id, "(RFC822)") 
    raw_email = data[0][1]
    msg = email.message_from_bytes(raw_email)
    body = str(msg.get_payload()[0].get_payload(decode=True))
#     print(msg['from'], msg['subject'])
    return(int(re.findall('(\d{6})', body)[0]))

driver = webdriver.Chrome()
# driver = webdriver.Firefox()

driver.get('https://www.freelancer.com/jobs/')
time.sleep(1)
login_button_1 = driver.find_element_by_xpath('/html/body/div[1]/header/div/div/div[2]/fl-login-signup-angular/a[1]')
login_button_1.click()

time.sleep(1)

cur.execute("SELECT id, password, username from users;")
r = random.choice(cur.fetchall())
print(r)

var_username = r[0]
var_pass = r[1]
username = driver.find_element_by_xpath('//*[@id="username"]')
password = driver.find_element_by_xpath('//*[@id="password"]')
username.clear()
username.send_keys(var_username)
password.clear()
password.send_keys(var_pass)
remember_me = driver.find_element_by_xpath('/html/body/fl-ui-modal/div/div[1]/div/div/fl-compiled/fl-login-signup-modal/div/div/fl-login/fl-login-form/div[2]/form[2]/fieldset/ol/li[3]/div/label')
# remember_me.clear()
remember_me.click()
login_button_2 = driver.find_element_by_xpath('//*[@id="login_btn"]')

login_button_2.click()
time.sleep(8)
give_me_code = driver.find_element_by_xpath('/html/body/fl-ui-modal/div/div[1]/div/div/fl-compiled/fl-login-signup-modal/div/div/fl-login/fl-totp-form/div[2]/div/form/fieldset/ol/li[2]/input')
give_me_code.send_keys(get_code_from_hotmail())

('akshay.kulkarni@pundir.solutions', 'fp4dTf', 'akulkarni227')


In [34]:
# try:


prev_count = 0
while True:
    driver.find_element_by_xpath('//*[@id="clear-btn"]').click()
    time.sleep(4)
    cur.execute("SELECT count(*) from projects;")
    count = cur.fetchone()[0]
    if count != prev_count:
        print('total : ', count)
        prev_count = count
    rows = driver.find_element_by_xpath('//*[@id="project_table"]/tbody')
    soup = BeautifulSoup(rows.get_attribute('innerHTML'), 'html.parser')
    for i in range(len(soup.find_all('tr'))):
        row = soup.find_all('tr')[i]
        if row.text == 'No projects or contests found.':
            print('break')
            break
        tags = []
        for j in range(len(row.find_all('span')[1].find_all('a'))):
            tags.append('\'' + row.find_all('span')[1].find_all('a')[j].text.strip() + '\'')
        url = row.find_all('a')[0]['href']
        project_id = int(row['id'])
        body = row.find_all('td')[0].find_all('p').pop().text.strip()
        bid_count = re.findall(r'\d+', row.find_all('td')[1].text.strip())[0]
        time_left = row.find_all('td')[2].find_all('small').pop().text
        money = row.find_all('td')[4].find_all('span')[0].text.strip()
        cur.execute("SELECT id FROM projects where id = %s;", (project_id,))
        if cur.fetchone() is None:
            cur.execute("INSERT INTO projects (id, bid_count, url, body, time_left, money, time) VALUES (%s, %s, %s, %s, %s, %s, %s)", (project_id, bid_count, url, body, time_left, money, time.time()))
            conn.commit()
            cur.execute("INSERT INTO tags (id, tag) VALUES (%s, ARRAY[%s])", (project_id, ', '.join(tags)))
            header = 'new'
            if('\'Excel\'' in tags):
                header = 'Excel'
            if('\'Python\'' in tags):
                header = 'Python'
            print(header, bid_count, url, money)
        else:
            cur.execute("SELECT bid_count from projects where id=%s", (project_id,))
            r = cur.fetchone()
            if r[0] != int(bid_count):
                cur.execute("UPDATE projects SET bid_count=%s, url=%s, body=%s, time_left=%s, money=%s WHERE id=%s", (bid_count, url, body, time_left, money, project_id))
                cur.execute("SELECT bid_count, time, url from projects where id=%s", (project_id,))
                r = cur.fetchone()
                print(r[0], time.time() - r[1], r[2])
    conn.commit()
    driver.refresh()
    time.sleep(4)
# except:
#     conn.commit()
#     print('success')

total :  329
new 3 https://www.freelancer.com/projects/graphic-design/ride-die-detailing/ ₹4545
new 0 https://www.freelancer.com/projects/internet-marketing/off-page-seo-techniques/ ₹2086 - ₹17392
new 7 https://www.freelancer.com/projects/graphic-design/greeting-cards-19408824/ ₹4818
new 2 https://www.freelancer.com/projects/php/fix-application-alert-system/ ₹1530
new 0 https://www.freelancer.com/contest/design-an-infographic-edt-1495553.html ₹5217
new 2 https://www.freelancer.com/projects/matlab-mathematica/drawing-roc-using-matlab/ ₹1739
25 130.34166860580444 https://www.freelancer.com/projects/graphic-design/need-infographic-designed-19408755/
6 168.35171604156494 https://www.freelancer.com/projects/php/view-downloaded-file-lighbox/
5 168.36136269569397 https://www.freelancer.com/projects/software-architecture/make-design-19408706/
7 262.36587953567505 https://www.freelancer.com/projects/graphic-design/build-school-management-android-app/
16 262.3714163303375 https://www.freelancer.

IndexError: list index out of range

In [31]:
row.text == 'No projects or contests found.'

True